In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/120-years-of-olympic-history-athletes-and-results/noc_regions.csv
/kaggle/input/120-years-of-olympic-history-athletes-and-results/athlete_events.csv


In [2]:
#Imports for data processing and modeling 
import sklearn as sk
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 

#read in the historical csv file as a dataframe, do some preprocessing to keep only important columns
df = pd.read_csv("/kaggle/input/120-years-of-olympic-history-athletes-and-results/athlete_events.csv")
df = df.sort_values("Sport")
df = df.drop(["ID", "Name", "Sex","Age", "Height", "Weight", "Team", "Games", "Season", "City", "Event"], axis=1)
df.loc[df["Medal"] == "Bronze","Medal"] = 1
df.loc[df["Medal"] == "Gold","Medal"] = 3
df.loc[df["Medal"] == "Silver","Medal"] = 2
df = df[df["Year"] >= 1976]
df = df.groupby(["Year","Sport","NOC"], as_index=False)["Medal"].sum()
merge = df.copy()

In [3]:
#label encoding categorical variables to form features
label_encoder = LabelEncoder()
df['NOC']= label_encoder.fit_transform(df['NOC']) 
df['Sport']= label_encoder.fit_transform(df['Sport']) 

In [4]:
#formatting the map for mapping country name to symbol for web app 
the_map = pd.read_csv("/kaggle/input/120-years-of-olympic-history-athletes-and-results/noc_regions.csv")
the_map= the_map.drop(["notes"], axis=1)
the_map["noc"] = the_map["NOC"]
the_map["country_name"] = the_map["region"]
the_map= the_map.drop(["region","NOC" ], axis=1)
file_path = '/kaggle/working/input/output.csv'
the_map.to_csv('country_mapper.csv',index=False)

In [5]:
#processing and formatting the prediction set
the_2024_results = merge[merge["Year"] == 2016]
the_2024_results.loc[:, 'Year'] = 2024
the_2024_results = the_2024_results.drop(["Medal"], axis=1)
copy = the_2024_results.copy()
label_encoder = LabelEncoder()
the_2024_results['NOC']= label_encoder.fit_transform(the_2024_results['NOC']) 
the_2024_results['Sport']= label_encoder.fit_transform(the_2024_results['Sport'])


In [6]:
#Making the train test split for the model given the data for experimentation
X= df[["Year", "Sport", "NOC"] ]
y=df[["Medal"] ]
X_train, X_test, y_train, y_test = train_test_split(X,y , 
                                   random_state=104,  
                                   test_size=.25,  
                                   shuffle=False) 
#prep for training the model
regressor = RandomForestRegressor(n_estimators=100, random_state=0, oob_score=True)
y_train= y_train["Medal"].to_numpy()
# Fit the model
regressor.fit(X_train, y_train)

RandomForestRegressor(oob_score=True, random_state=0)

In [7]:
#running predictions for olympics 2024
predictions_2024 = regressor.predict(the_2024_results)
predictions = pd.DataFrame(predictions_2024)

In [8]:
#merging the predictions with the formatted dataframe to pass as CSV into frontend
copy.reset_index(drop=True, inplace=True)
predictions.reset_index(drop=True, inplace=True)
concatenated_df = pd.concat([copy, predictions], axis=1)
concatenated_df.to_csv('predictions.csv',index=False)